In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)

%matplotlib inline

In [3]:
train = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/train.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/test.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

### 지하철, 버스정류장 이름 바꾸기

In [4]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수']

### 새로운 feature 추가

In [54]:
train['주차면수/총세대수'] = train['단지내주차면수']/train['총세대수']
train['공가수/주차면수'] = train['공가수']/train['단지내주차면수']
train['총임대세대수'] = train.groupby('단지코드')['전용면적별세대수'].transform(sum)
train['등록차량수/전용면적'] = train['등록차량수']/train['전용면적']
train['등록차량수/총임대세대수'] = train['등록차량수']/train['총임대세대수']
train['log_등록차량수'] = np.log1p(train['등록차량수'])

train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수,등록차량수/전용면적,등록차량수/총임대세대수,log_등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,3,25.553877,1.127778,6.923629
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,1,25.553877,1.127778,6.923629
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,4,19.545542,1.127778,6.923629
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,1,19.545542,1.127778,6.923629
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,2,19.545542,1.127778,6.923629
5,C2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,2,19.538017,1.127778,6.923629
6,C2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,3,19.538017,1.127778,6.923629
7,C2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000,214270,0.0,3.0,1425.0,1015.0,1.583333,0.026667,900,3,4,2,16.950568,1.127778,6.923629
8,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,4,6.123059,0.376147,5.327876
9,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,1.144954,0.027244,545,1,2,2,5.176768,0.376147,5.327876


In [6]:
test['주차면수/총세대수'] = test['단지내주차면수']/test['총세대수']
test['공가수/주차면수'] = test['공가수']/test['단지내주차면수']
test['총임대세대수'] = test.groupby('단지코드')['전용면적별세대수'].transform(sum)

test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,주차면수/총세대수,공가수/주차면수,총임대세대수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,0.905836,0.020498,754
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,0.905836,0.020498,754
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,0.905836,0.020498,754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0,0.691852,0.081370,670
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0,0.785340,0.150000,378
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0,0.785340,0.150000,378


### 지하철 결측치 처리

In [7]:
train['지하철'] = train['지하철'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   단지코드       2952 non-null   object 
 1   총세대수       2952 non-null   int64  
 2   임대건물구분     2952 non-null   object 
 3   지역         2952 non-null   object 
 4   공급유형       2952 non-null   object 
 5   전용면적       2952 non-null   float64
 6   전용면적별세대수   2952 non-null   int64  
 7   공가수        2952 non-null   float64
 8   자격유형       2952 non-null   object 
 9   임대보증금      2383 non-null   object 
 10  임대료        2383 non-null   object 
 11  지하철        2952 non-null   float64
 12  버스정류장      2948 non-null   float64
 13  단지내주차면수    2952 non-null   float64
 14  등록차량수      2952 non-null   float64
 15  주차면수/총세대수  2952 non-null   float64
 16  공가수/주차면수   2952 non-null   float64
 17  총임대세대수     2952 non-null   int64  
dtypes: float64(8), int64(3), object(7)
memory usage: 415.2+ KB
<class 'pandas.core.frame.Dat

(None, None)

### 버스정류장 결측치 처리

In [8]:
train['버스정류장'] = train['버스정류장'].fillna(2)

### 자격유형 결측치 처리

In [9]:
test.loc[196, '자격유형'] = 'A'
test.loc[258, '자격유형'] = 'C'

### QCUT

In [10]:
train['qcut_총세대수'] = pd.qcut(train['총세대수'], 5, labels=False)
train['qcut_단지내주차면수'] = pd.qcut(train['단지내주차면수'], 5, labels=False)
train['qcut_전용면적별세대수'] = pd.qcut(train['전용면적별세대수'], 5, labels=False)

In [11]:
test['qcut_총세대수'] = pd.qcut(test['총세대수'], 5, labels=False)
test['qcut_단지내주차면수'] = pd.qcut(test['단지내주차면수'], 5, labels=False)
test['qcut_전용면적별세대수'] = pd.qcut(test['전용면적별세대수'], 5, labels=False)

In [12]:
train.isnull().sum(), test.isnull().sum()

(단지코드               0
 총세대수               0
 임대건물구분             0
 지역                 0
 공급유형               0
 전용면적               0
 전용면적별세대수           0
 공가수                0
 자격유형               0
 임대보증금            569
 임대료              569
 지하철                0
 버스정류장              0
 단지내주차면수            0
 등록차량수              0
 주차면수/총세대수          0
 공가수/주차면수           0
 총임대세대수             0
 qcut_총세대수          0
 qcut_단지내주차면수       0
 qcut_전용면적별세대수      0
 dtype: int64,
 단지코드               0
 총세대수               0
 임대건물구분             0
 지역                 0
 공급유형               0
 전용면적               0
 전용면적별세대수           0
 공가수                0
 자격유형               0
 임대보증금            180
 임대료              180
 지하철                0
 버스정류장              0
 단지내주차면수            0
 주차면수/총세대수          0
 공가수/주차면수           0
 총임대세대수             0
 qcut_총세대수          0
 qcut_단지내주차면수       0
 qcut_전용면적별세대수      0
 dtype: int64)

## 데이터 분석

### 지역별 행복주택 중 자격유형 분포

In [41]:
train['지역'].value_counts()

경기도        621
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       151
광주광역시      141
경상북도       119
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
세종특별자치시     47
울산광역시       36
Name: 지역, dtype: int64

In [48]:
train[train['임대료']=='-']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,2
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,2
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,1
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,2
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0,4.297297,0.005660,370,0,4,3
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,-,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,3
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,-,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,-,0.0,3.0,673.0,645.0,0.851899,0.019316,790,2,3,2
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,-,-,0.0,4.0,1670.0,1153.0,0.863495,0.025749,1934,4,4,4


### 자격유형 A와 H의 관계는?

In [43]:
train[(train['공급유형']=='국민임대')&(train['지역']=='경기도')]['자격유형'].value_counts()

A    380
H     97
E     12
G      8
B      6
Name: 자격유형, dtype: int64

#### -> 다른지역의 A 와 H 학습 모델로 적용시켜보자

In [50]:
train[train['지역']=='경상남도']['지하철'].value_counts()

0.0    369
Name: 지하철, dtype: int64

In [30]:
train[train['공급유형']=='행복주택'].groupby('지역')['자격유형'].value_counts()

지역       자격유형
강원도      K        4
         J        1
경기도      J       36
         L       12
         N       12
경상남도     K       16
         L        8
경상북도     J        5
광주광역시    N       10
         J        8
         L        6
         M        2
         O        1
대구광역시    J        9
대전광역시    J        5
부산광역시    L        3
서울특별시    J       12
울산광역시    J       10
전라남도     N        8
         J        5
전라북도     J       10
제주특별자치도  J       13
충청북도     K       13
         L        4
Name: 자격유형, dtype: int64

### 자격유형이 K인 곳은?

In [32]:
train[train['자격유형']=='K']['지역'].value_counts()

경상남도    16
충청북도    13
강원도      4
Name: 지역, dtype: int64

#### 충북과 경남의 국민임대?

In [36]:
train[train['지역']=='충청북도']['공급유형'].value_counts()

국민임대         123
공공임대(10년)     22
행복주택          17
영구임대           1
Name: 공급유형, dtype: int64

#### -> 충남과 경남의 국민임대 - 행복주택을 학습시켜서 적용해도 되겠구나!

### 대구의 국민임대 (H)는?

In [38]:
train[(train['지역']=='대구광역시')&(train['자격유형']=='H')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수
2906,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.0,H,-,-,0.0,8.0,664.0,744.0,0.718615,0.0,916,3,3,4
2907,C2186,924,아파트,대구광역시,국민임대,29.34,38,0.0,H,-,-,0.0,8.0,664.0,744.0,0.718615,0.0,916,3,3,1
2908,C2186,924,아파트,대구광역시,국민임대,37.43,220,0.0,H,-,-,0.0,8.0,664.0,744.0,0.718615,0.0,916,3,3,4
2909,C2186,924,아파트,대구광역시,국민임대,46.51,188,0.0,H,25981000,248630,0.0,8.0,664.0,744.0,0.718615,0.0,916,3,3,4


In [47]:
train[(train['지역']=='대구광역시')&(train['자격유형']=='A')].shape

(75, 21)

In [24]:
train[train['임대보증금'].isna()].groupby('지역')['단지코드'].value_counts()

지역       단지코드 
강원도      C2212    11
         C1206    10
         C1722     6
         C1925     4
경상남도     C1790    30
         C2190    17
         C2109    15
         C2571    13
         C2416     8
         C1476     6
         C1698     4
         C1899     4
대전광역시    C1859    35
         C1616    19
         C2038    17
         C2258    17
         C1704    15
         C2621    10
         C1350     7
부산광역시    C1439    45
         C2132    43
         C2597    29
         C2034    26
         C1109    20
         C2289    20
         C2310    20
         C2135    17
제주특별자치도  C1775    15
         C1056    10
충청남도     C1874    27
         C1875    19
         C1004    17
         C1983    10
         C2644     3
Name: 단지코드, dtype: int64

In [15]:
pd.set_option('display.max_rows', None)
train[train['공급유형']=='공공분양']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,4
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,3
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,3
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0,1.167737,0.001222,1401,4,4,1


In [52]:
train[(train['지역']=='대전광역시')&(train['임대건물구분']=='아파트')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수
15,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,127350,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,4
16,C1407,1216,아파트,대전광역시,국민임대,30.99,68,13.0,A,15620000,127350,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,2
17,C1407,1216,아파트,대전광역시,국민임대,30.99,34,13.0,A,15620000,127350,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,1
18,C1407,1216,아파트,대전광역시,국민임대,41.11,148,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,3
19,C1407,1216,아파트,대전광역시,국민임대,41.39,74,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,2
20,C1407,1216,아파트,대전광역시,국민임대,41.58,70,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,2
21,C1407,1216,아파트,대전광역시,국민임대,41.58,70,13.0,A,27637000,164610,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,2
22,C1407,1216,아파트,대전광역시,국민임대,46.36,170,13.0,A,36048000,198250,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,4
23,C1407,1216,아파트,대전광역시,국민임대,46.36,170,13.0,A,36048000,198250,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,4
24,C1407,1216,아파트,대전광역시,국민임대,51.24,62,13.0,A,42056000,253520,1.0,1.0,1285.0,1064.0,1.056743,0.010117,1216,4,4,2


In [27]:
train[(train['임대건물구분']=='아파트')&(train['전용면적']>70)]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,등록차량수,주차면수/총세대수,공가수/주차면수,총임대세대수,qcut_총세대수,qcut_단지내주차면수,qcut_전용면적별세대수
1959,C1344,424,아파트,서울특별시,공공임대(분납),74.97,41,0.0,A,125326000,317660,0.0,3.0,467.0,509.0,1.101415,0.000000,300,0,2,2
1960,C1344,424,아파트,서울특별시,공공임대(분납),84.95,39,0.0,A,143545000,363840,0.0,3.0,467.0,509.0,1.101415,0.000000,300,0,2,1
1961,C1344,424,아파트,서울특별시,공공임대(분납),84.97,42,0.0,A,145264000,368190,0.0,3.0,467.0,509.0,1.101415,0.000000,300,0,2,2
1962,C1344,424,아파트,서울특별시,공공임대(분납),84.99,100,0.0,A,142744000,361800,0.0,3.0,467.0,509.0,1.101415,0.000000,300,0,2,3
2039,C1804,478,아파트,경기도,공공임대(10년),75.56,6,3.0,A,78957000,530130,0.0,3.0,629.0,951.0,1.315900,0.004769,26,1,2,1
2040,C1804,478,아파트,경기도,공공임대(10년),84.64,4,3.0,A,90237000,597820,0.0,3.0,629.0,951.0,1.315900,0.004769,26,1,2,0
2041,C1804,478,아파트,경기도,공공임대(10년),84.97,13,3.0,A,90237000,597820,0.0,3.0,629.0,951.0,1.315900,0.004769,26,1,2,1
2042,C1804,478,아파트,경기도,공공임대(10년),84.98,3,3.0,A,90237000,597820,0.0,3.0,629.0,951.0,1.315900,0.004769,26,1,2,0
2158,C1490,749,아파트,충청북도,공공임대(10년),74.42,86,24.0,A,26775000,378000,0.0,3.0,759.0,899.0,1.013351,0.031621,655,2,3,2
2159,C1490,749,아파트,충청북도,공공임대(10년),74.92,75,24.0,A,26775000,378000,0.0,3.0,759.0,899.0,1.013351,0.031621,655,2,3,2


### LabelEncoding

In [ ]:
## 우선 모두 진행
from sklearn.preprocessing import LabelEncoder

cate = ['임대건물구분', '지역', '자격유형', '공급유형']

encoder = LabelEncoder()

for one in cate:
    encoder.fit(train1[one])
    train1[one] = encoder.transform(train1[one])
    print(encoder.classes_)
    
    test1[one] = encoder.transform(test1[one])
    print(encoder.classes_)

### Age 정보 추가

In [ ]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])
for i in range(train.shape[0]):
    area = train.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
age_df

In [ ]:
train1 = pd.concat([train, age_df], axis=1)
train1

In [ ]:
age_df = pd.DataFrame(columns = ['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'])
for i in range(test.shape[0]):
    area = test.loc[i, '지역']
    age_df.loc[i] = age.loc[area]
age_df

In [ ]:
test1 = pd.concat([test, age_df], axis=1)
test1

## 모델 학습